

```
# EDA
```



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import plotly.offline as py
import plotly.express as px

import plotly.graph_objs as go
import plotly.figure_factory as ff
import cufflinks as cf

cf.set_config_file(offline=True, world_readable=False)

In [ ]:
import plotly.io as pio
pio.renderers.default = "colab"

# Introduction

In this lecture we examine the process of data cleaning and Exploratory Data Analysis (EDA).  Often you will acquire or even be given a collection of data in order to conduct some analysis or answer some questions. The first step in using that data is to ensure that it is in the correct form (cleaned) and that you understand its properties and limitations (EDA).  Often as you explore data through EDA you will identify additional transformations that may be required before the data is ready for analysis.

In this notebook we obtain crime data from the city of Berkeley's public records.  Ultimately, our goal might be to understand policing patterns but before we get there we must first clean and understand the data.

# Getting the Data

To begin this analysis we want to get data about crimes in Berkeley.  Remarkably, the city of Berkeley maintains an [Open Data Portal](https://data.cityofberkeley.info/) for citizens to access data about the city.  We will be examining the:

1. [Call Data](https://data.cityofberkeley.info/Public-Safety/Berkeley-PD-Calls-for-Service/k2nh-s5h5)
1. [Stop Data (NEW)](https://data.cityofberkeley.info/Public-Safety/Berkeley-PD-Stop-Data-NEW-/4tbf-3yt8)

Fortunately, this data is also relatively well document with detailed descriptions of what it contains.  Here are summaries of the fields in the data:

### Calls Data
<img src=https://drive.google.com/file/d/1gVrHtuC4nz_y-iwueWrDp-JB9JxuPNhb/view?usp=sharing width=800px />

### Stop Data
<img src=https://drive.google.com/file/d/1WR-y34BfO8ZTSyaf37iH2ZTHk6jM8R2O/view?usp=sharing width=800px />




## Most data has bad documentation:

Unfortunately, data is seldom well documented and when it is you may not be able to trust the documentation. It is therefore critical that when we download the data we investigate the fields and verify that it reflects the assumptions made in the documentation.



Helper function

In [ ]:
import requests
from pathlib import Path
import time

def fetch_and_cache(data_url, file, data_dir="data", force=False):
    """
    Download and cache a url and return the file object.

    data_url: the web address to download
    file: the file in which to save the results.
    data_dir: (default="data") the location to save the data
    force: if true the file is always re-downloaded

    return: The pathlib.Path object representing the file.
    """

    ### BEGIN SOLUTION
    data_dir = Path(data_dir)
    data_dir.mkdir(exist_ok = True)
    file_path = data_dir / Path(file)
    # If the file already exists and we want to force a download then
    # delete the file first so that the creation date is correct.
    if force and file_path.exists():
        file_path.unlink()
    if force or not file_path.exists():
        print('Downloading...', end=' ')
        resp = requests.get(data_url)
        with file_path.open('wb') as f:
            f.write(resp.content)
        print('Done!')
        last_modified_time = time.ctime(file_path.stat().st_mtime)
    else:
        last_modified_time = time.ctime(file_path.stat().st_mtime)
        print("Using cached version that was downloaded (UTC):", last_modified_time)
    return file_path
    ### END SOLUTION


#https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

def head(filename, lines=5):
    """
    Returns the first few lines of a file.

    filename: the name of the file to open
    lines: the number of lines to include

    return: A list of the first few lines from the file.
    """
    from itertools import islice
    with open(filename, "r") as f:
        return list(islice(f, lines))

In [ ]:
%load_ext autoreload
%autoreload 2

Occasionally, you will want to modify code that you have imported.  To reimport those modifications you can either use the python importlib library:

```python
from importlib import reload
reload(utils)
```

or use iPython magic which will intelligently import code when files change:

```python
%load_ext autoreload
%autoreload 2
```

## Downloading the Data

Notice that because I record how I got the data in the notebook, others can reproduce this experiment.  However, it is worth noting that **the data can change**.  We will want to pay attention to file timestamps.

In [ ]:
calls_file = fetch_and_cache("https://data.cityofberkeley.info/api/views/k2nh-s5h5/rows.csv?accessType=DOWNLOAD",
                "calls_for_service.csv", force=False)

Downloading... Done!


# Exploring the data

Now that we have obtained the data we want to understand its:

* **Structure** -- the "shape" of a data file
* **Granularity** -- how fine/coarse is each datum
* **Scope** -- how (in)complete is the data
* **Temporality** -- how is the data situated in time
* **Faithfulness** -- how well does the data capture "reality"



## Structure

Before we even begin to load the data it often helps to understand a little about the high-level structure:

1. How much data do I have?
1. How is it formatted?

### How big is the data?

I often like to start my analysis by getting a rough estimate of the size of the data.  This will help inform the tools I use and how I view the data.  If it is relatively small I might use a text editor or a spreadsheet to look at the data.  If it is larger, I might jump to more programmatic exploration or even used distributed computing tools.

However here we will use python tools to probe the file.

In [ ]:
import os
print(calls_file, "is",  os.path.getsize(calls_file) / 1e6, "MB")
print(stops_file, "is", os.path.getsize(stops_file) / 1e6, "MB")

data/calls_for_service.csv is 0.87466 MB
data/stops.json is 0.003694 MB


All the files are relatively small and we could comfortable examine them in a text editors.  (Personally, I like *sublime* or *emacs* but others may have a different *vi*ew.).

In listing the files I noticed that the names suggest that they are all text file formats:
* **CSV**: Comma separated values is a very standard table format.
* **JSON**: JavaScript Object Notation is a very standard semi-structured file format used to store nested data.

We will dive into the formats in a moment.  However because these are text data I might also want to investigate the number of lines which often correspond to records.

In [ ]:
with open(calls_file, "r") as f:
    print(calls_file, "is", sum(1 for l in f), "lines.")

data/calls_for_service.csv is 13457 lines.



### What is the file format?  (Can we trust extensions?)

We already noticed that the files end in `csv` and `json` which suggests that these are comma separated and javascript object files respectively.  However, we can't always rely on the naming as this is only a convention.  For example, here we picked the name of the file when downloading based on some hints in the URL.



**Often files will have incorrect extensions or no extension at all.**

Let's assume that these are text files (and do not contain binary encoded data) so we can print a "few lines" to get a better understanding of the file.

In [ ]:
print(calls_file, "======================")
with open(calls_file, "r") as f:
    for i in range(20):
        print(i, "\t", repr(f.readline()))

data/calls_for_service.csv ======================
0 	 'CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State\n'
1 	 '22036516,THEFT FROM AUTO,08/10/2022 12:00:00 AM,6:10:00 AM,LARCENY - FROM VEHICLE,3,10/13/2022 04:42:00 PM,"Berkeley, CA\n'
2 	 '(37.86988, -122.27054)",,Berkeley,CA\n'
3 	 '22027674,BURGLARY RESIDENTIAL,06/16/2022 12:00:00 AM,8:25:00 PM,BURGLARY - RESIDENTIAL,4,10/13/2022 04:42:00 PM,"1700 BLOCK OXFORD ST\n'
4 	 'Berkeley, CA\n'
5 	 '(37.87674, -122.26665)",1700 BLOCK OXFORD ST,Berkeley,CA\n'
6 	 '22023993,VANDALISM,05/25/2022 12:00:00 AM,8:00:00 AM,VANDALISM,3,10/13/2022 04:42:00 PM,"2400 BLOCK 8TH ST\n'
7 	 'Berkeley, CA\n'
8 	 '(37.86206, -122.29346)",2400 BLOCK 8TH ST,Berkeley,CA\n'
9 	 '22026547,DISTURBANCE,06/10/2022 12:00:00 AM,9:35:00 PM,DISORDERLY CONDUCT,5,10/13/2022 04:42:00 PM,"1300 BLOCK HEARST AVE\n'
10 	 'Berkeley, CA\n'
11 	 '(37.87133, -122.28656)",1300 BLOCK HEARST AVE,Berkeley,CA\n'
12 	 '22018126,ASSAULT/BATTERY FEL

Notice that I used the `repr` function to return the raw string with special characters.  This is helpful in deducing the file format.

### What are some observations about `Calls` data?

1. It appears to be in comma separated value (CSV) format.
1. First line contains the column headings.
1. There are lots of **new-line** `\n` characters:
    * at the ends of lines (delimiting records?)
    * *within records* as part of addresses.
1. There are **"quoted"** strings in the `Block_Location` column:
```
"2500 LE CONTE AVE
Berkeley, CA
(37.876965, -122.260544)"
```
these are going to be difficult.  What are the implications on our earlier line count calculations?

### What are some observations about `Stops` data?

This appears to be a fairly standard JSON file.  We notice that the file appears to contain a description of itself in a field called "meta" (which is presumably short for meta-data).  We will come back to this meta data in a moment but first let's quickly discuss the JSON file format.



### A quick note on JSON

[JSON (JavaScript Object Notation)](http://www.json.org) is a common format for exchanging complex structured and semi-structured data.

```javascript
{
    "field1": "value1",
    "field2": ["list", "of", "values"],
    "myfield3": {"is_recursive": true, "a null value": null}
}
```

A few key points:
* JSON is a recursive format in that JSON fields can also contain JSON objects
* JSON closely matches Python Dictionaries:
```python
d = {
    "field1": "value1",
    "field2": ["list", "of", "values"],
    "myfield3": {"is_recursive": True, "a null value": None}
}
print(d['myfield3'])
```
* Very common in web technologies (... JavaScript)
* Many languages have tools for loading and saving JSON objects




## Loading the Data

We will now attempt to load the data into python.  We will be using the Pandas dataframe library for basic tabular data analysis.  Fortunately, the Pandas library has some relatively sophisticated functions for loading data.


### Loading the Calls Data

Because the file appears to be a relatively well formatted CSV we will attempt to load it directly and allow the Pandas Library to deduce column headers.  (Always check that first row and column look correct after loading.)

In [ ]:
calls = pd.read_csv(calls_file, warn_bad_lines=True)
calls.head()

<ipython-input-16-89cbf5bc6161>:1: FutureWarning:

The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.





,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State
0,22036516,THEFT FROM AUTO,08/10/2022 12:00:00 AM,6:10:00 AM,LARCENY - FROM VEHICLE,3,10/13/2022 04:42:00 PM,"Berkeley, CA\n(37.86988, -122.27054)",NaN,Berkeley,CA
1,22027674,BURGLARY RESIDENTIAL,06/16/2022 12:00:00 AM,8:25:00 PM,BURGLARY - RESIDENTIAL,4,10/13/2022 04:42:00 PM,"1700 BLOCK OXFORD ST\nBerkeley, CA\n(37.87674,...",1700 BLOCK OXFORD ST,Berkeley,CA
2,22023993,VANDALISM,05/25/2022 12:00:00 AM,8:00:00 AM,VANDALISM,3,10/13/2022 04:42:00 PM,"2400 BLOCK 8TH ST\nBerkeley, CA\n(37.86206, -1...",2400 BLOCK 8TH ST,Berkeley,CA
3,22026547,DISTURBANCE,06/10/2022 12:00:00 AM,9:35:00 PM,DISORDERLY CONDUCT,5,10/13/2022 04:42:00 PM,"1300 BLOCK HEARST AVE\nBerkeley, CA\n(37.87133...",1300 BLOCK HEARST AVE,Berkeley,CA
4,22018126,ASSAULT/BATTERY FEL.,04/20/2022 12:00:00 AM,3:31:00 PM,ASSAULT,3,10/13/2022 04:42:00 PM,"3000 BLOCK SHATTUCK AVE\nBerkeley, CA\n(37.855...",3000 BLOCK SHATTUCK AVE,Berkeley,CA


How many records did we get?

In [ ]:
calls.shape[0]

4490

### Preliminary observations on the data?

1. `EVENTDT` -- Contain the incorrect time stamp
1. `EVENTTM` -- Contains the time in 24 hour format (What timezone?)
1. `CVDOW` -- Appears to be some encoding of the day of the week (see data documentation).
1. `InDbDate` -- Appears to be correctly formatted and appears pretty consistent in time.
1. **`Block_Location` -- Errr, what a mess!  newline characters, and Geocoordinates all merged!!  Fortunately, this field was "quoted" otherwise we would have had trouble parsing the file. (why?)**
1. `BLKADDR` -- This appears to be the address in Block Location.
1. `City` and `State` seem redundant given this is supposed to be the city of Berkeley dataset.


### Checking that the City and State fields are all Berkeley CA

We notice that there are city and state columns.  Since this is supposed to be data for the city of Berkeley these columns appear to be redundant.  Let's quickly compute the number of occurences of unique values for these two columns.

In [ ]:
calls.groupby(["City", "State"]).count()

,,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR
City,State,,,,,,,,,
Berkeley,CA,4490,4490,4490,4490,4490,4490,4490,4490,4476


### Decoding day of the week

According to the documentation `CVDOW=0` is Sunday, `CVDOW=1` is Monday, ...,  Therefore we can make a series to decode the day of the week for each record and join that series with the calls data.

In [ ]:
dow = pd.Series(["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"], name="Day")
dow

0       Sunday
1       Monday
2      Tuesday
3    Wednesday
4     Thursday
5       Friday
6     Saturday
Name: Day, dtype: object

In [ ]:
df_dow = pd.DataFrame(dow)
# Notice that I am dropping the column if it already exists to
# make it so I can run this cell more than once
calls = pd.merge(calls.drop(columns="Day", errors="ignore"),
         df_dow, left_on='CVDOW', right_index=True).sort_index()
calls

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day
0,22036516,THEFT FROM AUTO,08/10/2022 12:00:00 AM,6:10:00 AM,LARCENY - FROM VEHICLE,3,10/13/2022 04:42:00 PM,"Berkeley, CA\n(37.86988, -122.27054)",NaN,Berkeley,CA,Wednesday
1,22027674,BURGLARY RESIDENTIAL,06/16/2022 12:00:00 AM,8:25:00 PM,BURGLARY - RESIDENTIAL,4,10/13/2022 04:42:00 PM,"1700 BLOCK OXFORD ST\nBerkeley, CA\n(37.87674,...",1700 BLOCK OXFORD ST,Berkeley,CA,Thursday
2,22023993,VANDALISM,05/25/2022 12:00:00 AM,8:00:00 AM,VANDALISM,3,10/13/2022 04:42:00 PM,"2400 BLOCK 8TH ST\nBerkeley, CA\n(37.86206, -1...",2400 BLOCK 8TH ST,Berkeley,CA,Wednesday
3,22026547,DISTURBANCE,06/10/2022 12:00:00 AM,9:35:00 PM,DISORDERLY CONDUCT,5,10/13/2022 04:42:00 PM,"1300 BLOCK HEARST AVE\nBerkeley, CA\n(37.87133...",1300 BLOCK HEARST AVE,Berkeley,CA,Friday
4,22018126,ASSAULT/BATTERY FEL.,04/20/2022 12:00:00 AM,3:31:00 PM,ASSAULT,3,10/13/2022 04:42:00 PM,"3000 BLOCK SHATTUCK AVE\nBerkeley, CA\n(37.855...",3000 BLOCK SHATTUCK AVE,Berkeley,CA,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...
4485,22035888,FRAUD/FORGERY,08/01/2022 12:00:00 AM,4:30:00 PM,FRAUD,1,10/13/2022 04:42:00 PM,"900 BLOCK ADDISON ST\nBerkeley, CA\n(37.86699,...",900 BLOCK ADDISON ST,Berkeley,CA,Monday
4486,22018846,VEHICLE STOLEN,04/24/2022 12:00:00 AM,1:00:00 AM,MOTOR VEHICLE THEFT,0,10/13/2022 04:42:00 PM,"1800 BLOCK CAPISTRANO AVE\nBerkeley, CA\n(37.8...",1800 BLOCK CAPISTRANO AVE,Berkeley,CA,Sunday
4487,22043043,THEFT FROM AUTO,09/15/2022 12:00:00 AM,5:00:00 AM,LARCENY - FROM VEHICLE,4,10/13/2022 04:42:00 PM,"2300 BLOCK CURTIS ST\nBerkeley, CA\n(37.86476,...",2300 BLOCK CURTIS ST,Berkeley,CA,Thursday
4488,22028745,VEHICLE STOLEN,06/22/2022 12:00:00 AM,8:00:00 PM,MOTOR VEHICLE THEFT,3,10/13/2022 04:42:00 PM,"1700 BLOCK DERBY ST\nBerkeley, CA\n(37.85929, ...",1700 BLOCK DERBY ST,Berkeley,CA,Wednesday


### Cleaning Block Location

The block location contains the lat/lon coordinates and I might want to use these to analyze the location of each request.  Let's try to extract the GPS coordinates using regular expressions (we will cover regular expressions in future lectures):


In [ ]:
calls['Block_Location'].head(10)

0                 Berkeley, CA\n(37.86988, -122.27054)
1    1700 BLOCK OXFORD ST\nBerkeley, CA\n(37.87674,...
2    2400 BLOCK 8TH ST\nBerkeley, CA\n(37.86206, -1...
3    1300 BLOCK HEARST AVE\nBerkeley, CA\n(37.87133...
4    3000 BLOCK SHATTUCK AVE\nBerkeley, CA\n(37.855...
5    2400 BLOCK BLAKE ST\nBerkeley, CA\n(37.86397, ...
6    1800 BLOCK 6TH ST\nBerkeley, CA\n(37.87025, -1...
7    2400 BLOCK LE CONTE AVE\nBerkeley, CA\n(37.876...
8    2300 BLOCK DURANT AVE\nBerkeley, CA\n(37.86725...
9    2100 BLOCK 10TH ST\nBerkeley, CA\n(37.86746, -...
Name: Block_Location, dtype: object

In [ ]:
calls_lat_lon = (
    # Remove newlines
    calls['Block_Location'].str.replace("\n", "\t")
    # Extract Lat and Lon using regular expression
    .str.extract(".*\((?P<Lat>\d*\.\d*)\, (?P<Lon>-?\d*\.\d*)\)", expand=True)
)
calls_lat_lon.head(20)

,Lat,Lon
0,37.86988,-122.27054
1,37.87674,-122.26665
2,37.86206,-122.29346
3,37.87133,-122.28656
4,37.85507,-122.26677
5,37.86397,-122.26036
6,37.87025,-122.29843
7,37.87637,-122.26229
8,37.86725,-122.26326
9,37.86746,-122.29285


Not all the records have a lat and lon.  What fraction do have coordinates?

In [ ]:
(~calls_lat_lon.isnull()).mean()

Lat    1.0
Lon    1.0
dtype: float64

The following block of code joins the extracted Latitude and Longitude fields with the calls data.  Notice that we actually drop these fields before joining.  This is to enable repeated invocation of this cell even after the join has been completed.

In [ ]:
# Remove Lat and Lon if they already existed before (reproducible)
calls.drop(["Lat", "Lon"], axis=1, inplace=True, errors="ignore")
# Join in the the latitude and longitude data
calls = calls.merge(calls_lat_lon, left_index=True, right_index=True)
# calls[["Lat", "Lon"]] = calls_lat_lon
# calls.join(calls_lat_lon)
calls.head()

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,Lat,Lon
0,22036516,THEFT FROM AUTO,08/10/2022 12:00:00 AM,6:10:00 AM,LARCENY - FROM VEHICLE,3,10/13/2022 04:42:00 PM,"Berkeley, CA\n(37.86988, -122.27054)",NaN,Berkeley,CA,Wednesday,37.86988,-122.27054
1,22027674,BURGLARY RESIDENTIAL,06/16/2022 12:00:00 AM,8:25:00 PM,BURGLARY - RESIDENTIAL,4,10/13/2022 04:42:00 PM,"1700 BLOCK OXFORD ST\nBerkeley, CA\n(37.87674,...",1700 BLOCK OXFORD ST,Berkeley,CA,Thursday,37.87674,-122.26665
2,22023993,VANDALISM,05/25/2022 12:00:00 AM,8:00:00 AM,VANDALISM,3,10/13/2022 04:42:00 PM,"2400 BLOCK 8TH ST\nBerkeley, CA\n(37.86206, -1...",2400 BLOCK 8TH ST,Berkeley,CA,Wednesday,37.86206,-122.29346
3,22026547,DISTURBANCE,06/10/2022 12:00:00 AM,9:35:00 PM,DISORDERLY CONDUCT,5,10/13/2022 04:42:00 PM,"1300 BLOCK HEARST AVE\nBerkeley, CA\n(37.87133...",1300 BLOCK HEARST AVE,Berkeley,CA,Friday,37.87133,-122.28656
4,22018126,ASSAULT/BATTERY FEL.,04/20/2022 12:00:00 AM,3:31:00 PM,ASSAULT,3,10/13/2022 04:42:00 PM,"3000 BLOCK SHATTUCK AVE\nBerkeley, CA\n(37.855...",3000 BLOCK SHATTUCK AVE,Berkeley,CA,Wednesday,37.85507,-122.26677


We can now look at a few of the records that were missing latitude and longitude entries:

In [ ]:
calls[calls['Lat'].isnull()].head(10)

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,Lat,Lon


Are there any patterns to the missing data?

<br/><br/><br/>


---
## Loading the `stops.json` Data

Python has relatively good support for JSON data since it closely matches the internal python object model.  In the following cell we import the entire JSON datafile into a python dictionary.

In [ ]:
import io
import pandas as pd
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving stops.json to stops.json


In [ ]:
# df = pd.read_json(io.StringIO(uploaded.get('file.json').decode('utf-8')))

In [ ]:
import json

with open("data/stops.json", "r") as f:
  stops_json = json.load(f)


In [ ]:
import json

file_path = "data/stops.json"

try:
    with open(file_path, "r") as f:
        stops_json = json.load(f)
    print("JSON file loaded successfully.")
except FileNotFoundError:
    print(f"File not found: {file_path}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON in {file_path}: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Error decoding JSON in data/stops.json: Expecting value: line 1 column 1 (char 0)


In [ ]:
import json

file_path = "/content/gdrive/MyDrive/DataScienceCourse/Summer2023KS/lec/lec04/EDA/stops.json"

try:
    with open(file_path, "r") as f:
        content = f.read()
        if not content:
            print(f"The file {file_path} is empty.")
        else:
            stops_json = json.loads(content)
            print("JSON file loaded successfully.")
except FileNotFoundError:
    print(f"File not found: {file_path}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON in {file_path}: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


JSON file loaded successfully.


The `stops_json` variable is now a dictionary encoding the data in the file:

In [ ]:
stops_json

{'meta': {'view': {'id': '4tbf-3yt8',
   'name': 'Berkeley PD - Stop Data (NEW)',
   'attribution': 'City of Berkeley Police Department',
   'averageRating': 0,
   'category': 'Public Safety',
   'createdAt': 1588602591,
   'description': 'This data was extracted from the Department’s Public Safety Server and covers data beginning January 26, 2015.  On January 26, 2015 the department began collecting data pursuant to General Order B-4 (issued December 31, 2014). Under that Order, officers were required to provide certain data after making any detention (vehicle, bicycle, pedestrian, suspicious auto).  This dataset provides information about detentions, including the race, sex, age range, of the person detained; the reason for the stop; the type of enforcement taken (if any), and whether or not a search was conducted.  Also provided are the date, time, location of the detention, as well as the incident number and call for service type.',
   'displayType': 'table',
   'downloadCount': 48

In [ ]:
type(stops_json)

dict

---
<br/><br/><br/>

### We can now examine what keys are in the top level json object

We can list the keys to determine what data is stored in the object.

In [ ]:
stops_json.keys()

dict_keys(['meta', 'data'])

#### Observation

The JSON dictionary contains a `meta` key which likely refers to meta data (data about the data).  Meta data often maintained with the data and can be a good source of additional information.

---
<br/><br/><br/>

## Digging into Meta Data

We can investigate the meta data further by examining the keys associated with the metadata.

In [ ]:
stops_json['meta'].keys()

dict_keys(['view'])

The `meta` key contains another dictionary called `view`.  This likely refers to meta-data about a particular "view" of some underlying database.  We will learn more about views as we study SQL later in the class.    

In [ ]:
stops_json['meta']['view'].keys()

dict_keys(['id', 'name', 'attribution', 'averageRating', 'category', 'createdAt', 'description', 'displayType', 'downloadCount', 'hideFromCatalog', 'hideFromDataJson', 'licenseId', 'newBackend', 'numberOfComments', 'oid', 'provenance', 'publicationAppendEnabled', 'publicationDate', 'publicationGroup', 'publicationStage', 'rowsUpdatedAt', 'rowsUpdatedBy', 'tableId', 'totalTimesRated', 'viewCount', 'viewLastModified', 'viewType', 'approvals', 'columns', 'grants', 'license', 'metadata', 'owner', 'query', 'rights', 'tableAuthor', 'tags', 'flags'])

Notice that this a nested/recursive data structure.  As we dig deeper we reveal more and more keys and the corresponding data:

```
meta
|-> data
    | ... (haven't explored yet)
|-> view
    | -> id
    | -> name
    | -> attribution
    ...
```

There is a key called description in the view sub dictionary.  This likely contains a description of the data:

In [ ]:
print(stops_json['meta']['view']['description'])

This data was extracted from the Department’s Public Safety Server and covers data beginning January 26, 2015.  On January 26, 2015 the department began collecting data pursuant to General Order B-4 (issued December 31, 2014). Under that Order, officers were required to provide certain data after making any detention (vehicle, bicycle, pedestrian, suspicious auto).  This dataset provides information about detentions, including the race, sex, age range, of the person detained; the reason for the stop; the type of enforcement taken (if any), and whether or not a search was conducted.  Also provided are the date, time, location of the detention, as well as the incident number and call for service type.


---
<br/><br/><br/>

### Columns Meta data

Another potentially useful key in the meta data dictionary is the `columns`.  This returns a list:

In [ ]:
type(stops_json['meta']['view']['columns'])

list

We can browse summary data in the list using python:

In [ ]:
for c in stops_json['meta']['view']['columns']:
    top = ""
    items = ""
    if "cachedContents" in c:
        top = c["cachedContents"]["top"]
        for i in top:
            items = items + "\n\t" + i["item"] + ":" + i["count"]
    print(c["name"], ":", items)

sid : 
id : 
position : 
created_at : 
created_meta : 
updated_at : 
updated_meta : 
meta : 
CreateDatetime : 
	2015-06-11T06:48:26.000:10
	2020-02-13T08:10:43.000:6
	2017-01-26T11:10:06.000:6
	2020-01-07T08:15:40.000:6
	2016-07-03T23:45:04.000:5
	2015-02-01T19:33:13.000:5
	2016-06-11T19:55:36.000:5
	2015-11-16T07:19:28.000:5
	2016-04-19T00:50:19.000:5
	2016-03-25T01:13:14.000:5
	2017-01-06T00:25:16.000:5
	2016-12-22T23:58:59.000:5
	2016-07-19T02:51:56.000:4
	2015-11-09T22:17:36.000:4
	2015-12-19T02:11:34.000:4
	2016-07-21T02:03:22.000:4
	2017-06-29T02:15:29.000:4
	2018-07-02T21:36:29.000:4
	2016-05-25T01:20:06.000:4
	2016-05-01T01:23:53.000:4
IncidentNumber : 
	2020-00001049:6
	2020-00008547:6
	2015-00033565:5
	2016-00039405:5
	2016-00017339:5
	2016-00075241:5
	2016-00034574:5
	2017-00000828:5
	2015-00033576:5
	2015-00067321:5
	2015-00006347:5
	2016-00022806:5
	2016-00030608:4
	2019-00025759:4
	2016-00049625:4
	2018-00010987:4
	2015-00033137:4
	2017-00045853:4
	2016-00042412:4
	2017-0


### Observations?

1. The above meta data tells us a lot about the columns in the data including column names, potential data anomalies, and a basic statistic.
1. The old version of this data included descriptions which would be useful in loading and working with the data.
1. JSON makes it easier (than CSV) to create "self-documented data".
1. Self documenting data can be helpful since it maintains it's own description and these descriptions are more likely to be updated as data changes.


### Examining the Data Field

We can look at a few entires in the data field


In [ ]:
for i in range(3):
    print(i, "\t", stops_json['data'][i])

0 	 ['row-2j6s.qe6z_jtkn', '00000000-0000-0000-D599-E1C6017B6966', 0, 1589533219, None, 1589533219, None, '{ }', '2017-05-02T20:50:29', '2017-00024695', '1275 WALNUT ST', 'BERKELEY', '37.88452086', '-122.268652364', '1194', 'White', 'Female', '30-39', 'Investigation', 'Warning', 'No Search']
1 	 ['row-79rz~ucps-35r6', '00000000-0000-0000-B83C-7A2ADDC55727', 0, 1598605226, None, 1598605226, None, '{ }', '2018-02-20T18:42:04', '2018-00010451', '2176 KITTREDGE ST', 'BERKELEY', '37.868233788', '-122.266394615', '1194', 'White', 'Male', '>40', 'Investigation', 'Other', 'No Search']
2 	 ['row-tmr9_n9ye.6bzh', '00000000-0000-0000-3120-DC62279E5799', 0, 1598605226, None, 1598605226, None, '{ }', '2020-03-04T16:18:11', '2020-00012764', '1414 UNIVERSITY AVE', 'BERKELEY', '37.869690714', '-122.283865643', 'T', 'Black', 'Female', '>40', 'Traffic', 'Citation', 'No Search']


## Building a Dataframe from JSON

In the following block of code we:
1. Translate the JSON records into a dataframe
1. Remove columns that have no metadata description.  This would be a bad idea in general but here we remove these columns since the above analysis suggests that they are unlikely to contain useful information.
1. Examine the top of the table

In [ ]:
# Load the data from JSON and assign column titles
stops = pd.DataFrame(
    stops_json['data'],
    columns=[c['name'] for c in stops_json['meta']['view']['columns']])

stops.head()

,sid,id,position,created_at,created_meta,updated_at,updated_meta,meta,CreateDatetime,IncidentNumber,...,City,Lat,Lon,CallType,Race,Gender,Age,Reason,Enforcement,Car Search
0,row-2j6s.qe6z_jtkn,00000000-0000-0000-D599-E1C6017B6966,0,1589533219,None,1589533219,None,{ },2017-05-02T20:50:29,2017-00024695,...,BERKELEY,37.88452086,-122.268652364,1194,White,Female,30-39,Investigation,Warning,No Search
1,row-79rz~ucps-35r6,00000000-0000-0000-B83C-7A2ADDC55727,0,1598605226,None,1598605226,None,{ },2018-02-20T18:42:04,2018-00010451,...,BERKELEY,37.868233788,-122.266394615,1194,White,Male,>40,Investigation,Other,No Search
2,row-tmr9_n9ye.6bzh,00000000-0000-0000-3120-DC62279E5799,0,1598605226,None,1598605226,None,{ },2020-03-04T16:18:11,2020-00012764,...,BERKELEY,37.869690714,-122.283865643,T,Black,Female,>40,Traffic,Citation,No Search
3,row-cjve_43yp_s8ec,00000000-0000-0000-4EEF-F940D151C603,0,1598605226,None,1598605226,None,{ },2018-12-04T15:11:14,2018-00067166,...,BERKELEY,37.867138358,-122.268229602,1196,Hispanic,Male,30-39,Traffic,Warning,No Search
4,row-wxqp_6mgh-v4r5,00000000-0000-0000-8B94-7D86EB47C51D,0,1589533219,None,1589533219,None,{ },2020-05-13T09:37:53,2020-00023435,...,BERKELEY,37.865779797,-122.267576002,1194,Black,Male,>40,Traffic,Citation,No Search


In [ ]:
stops.columns

Index(['sid', 'id', 'position', 'created_at', 'created_meta', 'updated_at',
       'updated_meta', 'meta', 'CreateDatetime', 'IncidentNumber', 'Address',
       'City', 'Lat', 'Lon', 'CallType', 'Race', 'Gender', 'Age', 'Reason',
       'Enforcement', 'Car Search'],
      dtype='object')

Too many columns.  Let's ask pandas to show us more.  Be careful, showing too much could break your notebook.

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [ ]:
stops.head(10)

,sid,id,position,created_at,created_meta,updated_at,updated_meta,meta,CreateDatetime,IncidentNumber,Address,City,Lat,Lon,CallType,Race,Gender,Age,Reason,Enforcement,Car Search
0,row-2j6s.qe6z_jtkn,00000000-0000-0000-D599-E1C6017B6966,0,1589533219,None,1589533219,None,{ },2017-05-02T20:50:29,2017-00024695,1275 WALNUT ST,BERKELEY,37.88452086,-122.268652364,1194,White,Female,30-39,Investigation,Warning,No Search
1,row-79rz~ucps-35r6,00000000-0000-0000-B83C-7A2ADDC55727,0,1598605226,None,1598605226,None,{ },2018-02-20T18:42:04,2018-00010451,2176 KITTREDGE ST,BERKELEY,37.868233788,-122.266394615,1194,White,Male,>40,Investigation,Other,No Search
2,row-tmr9_n9ye.6bzh,00000000-0000-0000-3120-DC62279E5799,0,1598605226,None,1598605226,None,{ },2020-03-04T16:18:11,2020-00012764,1414 UNIVERSITY AVE,BERKELEY,37.869690714,-122.283865643,T,Black,Female,>40,Traffic,Citation,No Search
3,row-cjve_43yp_s8ec,00000000-0000-0000-4EEF-F940D151C603,0,1598605226,None,1598605226,None,{ },2018-12-04T15:11:14,2018-00067166,2300 SHATTUCK AVE,BERKELEY,37.867138358,-122.268229602,1196,Hispanic,Male,30-39,Traffic,Warning,No Search
4,row-wxqp_6mgh-v4r5,00000000-0000-0000-8B94-7D86EB47C51D,0,1589533219,None,1589533219,None,{ },2020-05-13T09:37:53,2020-00023435,SHATTUCK AVE / CHANNING WAY,BERKELEY,37.865779797,-122.267576002,1194,Black,Male,>40,Traffic,Citation,No Search
5,row-mf49~8gwk.k6g5,00000000-0000-0000-43A3-D477F3C07872,0,1598605226,None,1598605226,None,{ },2019-04-19T04:53:10,2019-00020838,3400 ADELINE ST,BERKELEY,37.846549841,-122.273012707,1196,Other,Male,18-29,Investigation,Warning,No Search
6,row-zks9-32fu.such,00000000-0000-0000-0073-AF5810C4FB2A,0,1598605226,None,1598605226,None,{ },2016-11-20T04:28:07,2016-00068586,1250 UNIVERSITY AVE,BERKELEY,37.869522906,-122.287045202,1196,Black,Male,>40,Investigation,Warning,No Search
7,row-5rst-vhd2-pwxf,00000000-0000-0000-1169-8943ED935A5C,0,1598605226,None,1598605226,None,{ },2020-01-26T23:35:54,2020-00004974,735 ASHBY AVE,BERKELEY,37.850242359,-122.294539787,T,Other,Female,>40,Traffic,Warning,No Search
8,row-44ny.assv-86xh,00000000-0000-0000-EEBF-C9FBD906B2FF,0,1598605226,None,1598605226,None,{ },2016-12-19T01:40:47,2016-00074453,2887 COLLEGE AVE,BERKELEY,37.858828873,-122.252913485,1194,White,Male,30-39,Investigation,Warning,No Search
9,row-cza5_4r84.ajdc,00000000-0000-0000-1729-EE9C56DE857D,0,1598605226,None,1598605226,None,{ },2017-04-15T22:30:50,2017-00021046,SACRAMENTO ST / HEARST AVE,BERKELEY,37.87214977,-122.282132237,T,Asian,Male,18-29,Traffic,Citation,No Search


## Preliminary Observations

What do we observe so far?

We observe:
1. The `Incident Number` appears to have the year encoded in it - we could potentially use this as a validation check.  
1. The `created_at` and `updated_at` Fields look like they are in milliseconds since January 1, 1970.
1. The `CreateDatetime` Field looks to be formatted in YYYY-MM-DDTHH:MM:SS.  I am guessing T means "Time".
1. The `Age` Field has variable size brackets: 18-29, 30-39, >40.
1. The definition of `CallType` can be found in [Berkeley Police DepartmentCall-Incident Types](https://www.cityofberkeley.info/uploadedFiles/Police/Level_3_-_General/Call-Incident%20Types.pdf): 1194-Pedestrian Stop, 1196-Suspicious Vehicle Stop, T-Traffic Stop
     
Recall the description:

### Stop Data
<img src="stops_desc.png" width=800px />



---

<br/><br/><br/>

# EDA

Now that we have loaded our various data files.  Let's try to understand a bit more about the data by examining properties of individual fields.

In [ ]:
calls.head()

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,Lat,Lon,timestamp,hour_of_day
0,22036516,THEFT FROM AUTO,08/10/2022 12:00:00 AM,6:10:00 AM,LARCENY - FROM VEHICLE,3,10/13/2022 04:42:00 PM,"Berkeley, CA\n(37.86988, -122.27054)",NaN,Berkeley,CA,Wednesday,37.86988,-122.27054,2022-08-10 06:10:00,6.166667
1,22027674,BURGLARY RESIDENTIAL,06/16/2022 12:00:00 AM,8:25:00 PM,BURGLARY - RESIDENTIAL,4,10/13/2022 04:42:00 PM,"1700 BLOCK OXFORD ST\nBerkeley, CA\n(37.87674,...",1700 BLOCK OXFORD ST,Berkeley,CA,Thursday,37.87674,-122.26665,2022-06-16 20:25:00,20.416667
2,22023993,VANDALISM,05/25/2022 12:00:00 AM,8:00:00 AM,VANDALISM,3,10/13/2022 04:42:00 PM,"2400 BLOCK 8TH ST\nBerkeley, CA\n(37.86206, -1...",2400 BLOCK 8TH ST,Berkeley,CA,Wednesday,37.86206,-122.29346,2022-05-25 08:00:00,8.000000
3,22026547,DISTURBANCE,06/10/2022 12:00:00 AM,9:35:00 PM,DISORDERLY CONDUCT,5,10/13/2022 04:42:00 PM,"1300 BLOCK HEARST AVE\nBerkeley, CA\n(37.87133...",1300 BLOCK HEARST AVE,Berkeley,CA,Friday,37.87133,-122.28656,2022-06-10 21:35:00,21.583333
4,22018126,ASSAULT/BATTERY FEL.,04/20/2022 12:00:00 AM,3:31:00 PM,ASSAULT,3,10/13/2022 04:42:00 PM,"3000 BLOCK SHATTUCK AVE\nBerkeley, CA\n(37.855...",3000 BLOCK SHATTUCK AVE,Berkeley,CA,Wednesday,37.85507,-122.26677,2022-04-20 15:31:00,15.516667


---
<br/><br/><br/>

### Are Case Numbers unique?

Case numbers are probably used internally to track individual cases and my reference other data we don't have access to.  However, it is possible that multiple calls could be associated with the same case.  Let's see if the case numbers are all unique.

In [ ]:
print("There are", calls['CASENO'].unique().shape[0], "unique case numbers.")
print("There are", calls.shape[0], "calls in the table.")

There are 4490 unique case numbers.
There are 4490 calls in the table.


Are case numbers assigned consecutively.  

In [ ]:
calls['CASENO'].sort_values().reset_index(drop=True).iplot(
    yTitle="Case Number", xTitle="Location in File")

I like to use interactive plotting tools so I can hover the mouse over the plot and read the values.  The cufflinks library adds plotly support to Pandas.  

### What might we be observing?

It looks like there are three discrete regions - an initial value, a consecutive increasing range, a slight gap(!) and another consecutive increasing range.

In [ ]:
calls['CASENO'].sort_values().head()

2747    22001097
3432    22007237
1103    22009526
3801    22017398
4079    22017403
Name: CASENO, dtype: int64

Let's look at row 4121.

In [ ]:
calls.iloc[[4121]]

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,Lat,Lon,timestamp,hour_of_day
4121,22090846,BURGLARY AUTO,07/17/2022 12:00:00 AM,2:00:00 PM,BURGLARY - VEHICLE,0,10/13/2022 04:42:00 PM,"2900 BLOCK COLLEGE AVE\nBerkeley, CA\n(37.8582...",2900 BLOCK COLLEGE AVE,Berkeley,CA,Sunday,37.85824,-122.25317,2022-07-17 14:00:00,14.0


This record looks very anomalous as it is missing values for multiple important fields.

---
<br/><br/><br/>

## Examining the Date

Let's dig into the date in which events were recorded.  Notice in this data we have several pieces of date/time information (this is not uncommon):
1. **`EVENTDT`**: This contains the date the event took place.  While it has time information the time appears to be `00:00:00`.  
1. **`EVENTTM`**: This contains the time at which the event took place.
1. **`InDbDate`**: This appears to be the date at which the data was entered in the database.  

In [ ]:
calls.head(3)

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,Lat,Lon,timestamp,hour_of_day
0,22036516,THEFT FROM AUTO,08/10/2022 12:00:00 AM,6:10:00 AM,LARCENY - FROM VEHICLE,3,10/13/2022 04:42:00 PM,"Berkeley, CA\n(37.86988, -122.27054)",NaN,Berkeley,CA,Wednesday,37.86988,-122.27054,2022-08-10 06:10:00,6.166667
1,22027674,BURGLARY RESIDENTIAL,06/16/2022 12:00:00 AM,8:25:00 PM,BURGLARY - RESIDENTIAL,4,10/13/2022 04:42:00 PM,"1700 BLOCK OXFORD ST\nBerkeley, CA\n(37.87674,...",1700 BLOCK OXFORD ST,Berkeley,CA,Thursday,37.87674,-122.26665,2022-06-16 20:25:00,20.416667
2,22023993,VANDALISM,05/25/2022 12:00:00 AM,8:00:00 AM,VANDALISM,3,10/13/2022 04:42:00 PM,"2400 BLOCK 8TH ST\nBerkeley, CA\n(37.86206, -1...",2400 BLOCK 8TH ST,Berkeley,CA,Wednesday,37.86206,-122.29346,2022-05-25 08:00:00,8.000000


When Pandas loads more complex fields like dates it will often load them as strings:

In [ ]:
calls["EVENTDT"][0]

'08/10/2022 12:00:00 AM'

We will want to convert these to dates.  Pandas has a fairly sophisticated function `pd.to_datetime` which is capable of guessing reasonable conversions of dates to date objects.

In [ ]:
dates = pd.to_datetime(calls["EVENTDT"])
dates[0]

Timestamp('2022-08-10 00:00:00')

We can verify that the translations worked by looking at a few dates:

In [ ]:
pd.DataFrame(dict(transformed=dates, original=calls["EVENTDT"])).head()

,transformed,original
0,2022-08-10,08/10/2022 12:00:00 AM
1,2022-06-16,06/16/2022 12:00:00 AM
2,2022-05-25,05/25/2022 12:00:00 AM
3,2022-06-10,06/10/2022 12:00:00 AM
4,2022-04-20,04/20/2022 12:00:00 AM


We can also extract the time field:

In [ ]:
times = pd.to_datetime(calls["EVENTTM"]).dt.time
times.head()

0    06:10:00
1    20:25:00
2    08:00:00
3    21:35:00
4    15:31:00
Name: EVENTTM, dtype: object

To combine the correct date and correct time field we use the built-in python datetime combine function.

In [ ]:
from datetime import datetime
timestamps = pd.concat([dates, times], axis=1).apply(
    lambda r: datetime.combine(r['EVENTDT'], r['EVENTTM']), axis=1)
timestamps.head()

0   2022-08-10 06:10:00
1   2022-06-16 20:25:00
2   2022-05-25 08:00:00
3   2022-06-10 21:35:00
4   2022-04-20 15:31:00
dtype: datetime64[ns]

We now updated calls to contain this additional informations:

In [ ]:
calls['timestamp'] = timestamps
calls.head()

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,Lat,Lon,timestamp
0,22036516,THEFT FROM AUTO,08/10/2022 12:00:00 AM,6:10:00 AM,LARCENY - FROM VEHICLE,3,10/13/2022 04:42:00 PM,"Berkeley, CA\n(37.86988, -122.27054)",NaN,Berkeley,CA,Wednesday,37.86988,-122.27054,2022-08-10 06:10:00
1,22027674,BURGLARY RESIDENTIAL,06/16/2022 12:00:00 AM,8:25:00 PM,BURGLARY - RESIDENTIAL,4,10/13/2022 04:42:00 PM,"1700 BLOCK OXFORD ST\nBerkeley, CA\n(37.87674,...",1700 BLOCK OXFORD ST,Berkeley,CA,Thursday,37.87674,-122.26665,2022-06-16 20:25:00
2,22023993,VANDALISM,05/25/2022 12:00:00 AM,8:00:00 AM,VANDALISM,3,10/13/2022 04:42:00 PM,"2400 BLOCK 8TH ST\nBerkeley, CA\n(37.86206, -1...",2400 BLOCK 8TH ST,Berkeley,CA,Wednesday,37.86206,-122.29346,2022-05-25 08:00:00
3,22026547,DISTURBANCE,06/10/2022 12:00:00 AM,9:35:00 PM,DISORDERLY CONDUCT,5,10/13/2022 04:42:00 PM,"1300 BLOCK HEARST AVE\nBerkeley, CA\n(37.87133...",1300 BLOCK HEARST AVE,Berkeley,CA,Friday,37.87133,-122.28656,2022-06-10 21:35:00
4,22018126,ASSAULT/BATTERY FEL.,04/20/2022 12:00:00 AM,3:31:00 PM,ASSAULT,3,10/13/2022 04:42:00 PM,"3000 BLOCK SHATTUCK AVE\nBerkeley, CA\n(37.855...",3000 BLOCK SHATTUCK AVE,Berkeley,CA,Wednesday,37.85507,-122.26677,2022-04-20 15:31:00


### What time range does the data represent

In [ ]:
calls['timestamp'].min()

Timestamp('2022-04-16 00:00:00')

In [ ]:
calls['timestamp'].max()

Timestamp('2022-10-12 20:30:00')

---
<br/><br/><br/>


### Are there any other interesting temporal patterns

Do more calls occur on a particular day of the week?

In [ ]:
dow = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
calls.groupby('Day')['CASENO'].count()[dow].iplot(kind='bar', yTitle="Count")

### How about temporal patterns within a day?

In [ ]:
calls['hour_of_day'] = (
    calls['timestamp'].dt.hour * 60 + calls['timestamp'].dt.minute ) / 60.

In [ ]:
py.iplot(ff.create_distplot([calls['hour_of_day']],group_labels=["Hour"],bin_size=1, show_rug=False))

### Observations?

In the above plot we see the standard pattern of limited activity early in the morning around here 6:00AM.

### Stratified Analysis

To better understand the time of day a report occurs we could stratify the analysis by the day of the week.  To do this we will use box plots.  


In [ ]:
px.violin(calls.sort_values("CVDOW"), y="hour_of_day", x="Day", box=True, points="all", hover_name="CVLEGEND")

#### Observations?

There are no very clear patterns here.  However it does appear that weekends have more calls later into the night.

---
<br/><br/><br/>

### Examining the Event

We also have data about the different kinds of crimes being reported

In [ ]:
calls.head()

### The Offense Field

The Offense field appears to contain the specific crime being reported.  As nominal data we might want to see a summary constructed by computing counts of each offense type:

In [ ]:
calls['OFFENSE'].value_counts().iplot(kind="bar")

#### Observations?

Car burglary and misdemeanor theft seem to be the most common crimes with many other types of crimes occurring rarely.

---
<br/><br/><br/>

### CVLEGEND

The CVLEGEND field provides the broad category of crime and is a good mechanism to group potentially similar crimes.

In [ ]:
calls['CVLEGEND'].value_counts().iplot(kind="bar")

Notice that when we group by the crime time we see that **larceny** emerges as one of the top crimes.  Larceny is essentially stealing -- taking someone else stuff without force.

### Stratified Analysis of Time of Day by CVLEGEND

View the crime time periods broken down by crime type:

In [ ]:
boxes = [(len(df), go.Violin(y=df["hour_of_day"], name=i)) for (i, df) in calls.groupby("CVLEGEND")]
py.iplot([r[1] for r in sorted(boxes, key=lambda x:x[0], reverse=True)])

In [ ]:
py.iplot(ff.create_distplot([
    calls[calls['CVLEGEND'] == "NOISE VIOLATION"]['hour_of_day'],
    calls[calls['CVLEGEND'] == "DRUG VIOLATION"]['hour_of_day'],
    calls[calls['CVLEGEND'] == "LIQUOR LAW VIOLATION"]['hour_of_day'],
    calls[calls['CVLEGEND'] == "FRAUD"]['hour_of_day']
],
    group_labels=["Noise Violation", "Drug Violation", "Liquor Violation", "Fraud"],
    ))


## Examining Location information

Let's examine the geographic data (latitude and longitude).  Recall that we had some missing values.  Let's look at the behavior of these missing values according to crime type.

In [ ]:
calls["missing_lat_lon"] = calls[['Lat', 'Lon']].isnull().any(axis=1)
calls.groupby("CVLEGEND")['missing_lat_lon'].mean().sort_values().iplot(kind="barh")

### Observations?

There is a clear bias towards sex crimes and drug violations that is not present in the original data.  Therefore we should be careful when dropping missing values!

---
<br/><br/><br/>

### Examine data geographically


In [ ]:
import folium
import folium.plugins # The Folium Javascript Map Library

SF_COORDINATES = (37.87, -122.28)
sf_map = folium.Map(location=SF_COORDINATES, zoom_start=13)
locs = calls[['Lat', 'Lon']].astype('float').dropna().to_numpy()
heatmap = folium.plugins.HeatMap(locs.tolist(), radius = 10)
sf_map.add_child(heatmap)

### Questions

1. Why are all the calls located on the street and at often at intersections?


In [ ]:
cluster = folium.plugins.MarkerCluster()
for _, r in calls[['Lat', 'Lon', 'CVLEGEND']].tail(1000).dropna().iterrows():
    cluster.add_child(
        folium.Marker([float(r["Lat"]), float(r["Lon"])], popup=r['CVLEGEND']))

sf_map = folium.Map(location=SF_COORDINATES, zoom_start=13)
sf_map.add_child(cluster)
sf_map